# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME") # gpt-4o-mini
openai = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [06:33<00:00, 78.61s/it]


In [5]:
len(deals)

50

In [6]:
deals[44].describe()

"Title: DeWalt 20V Lithium-ion Battery 2-Pack for $69 + free shipping\nDetails: It's the best price we could find by $50. Buy Now at Lowe's\nFeatures: \nURL: https://www.dealnews.com/products/De-Walt/De-Walt-20-V-Lithium-ion-Battery-2-Pack/485597.html?iref=rss-c196"

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [9]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Samsung S90D Series QN42S90DAEXZA 42" 4K HDR 144Hz OLED Smart TV for $1,100 + free shipping
Details: That's $300 off, and the best price we could find for this OLED TV. Buy Now at Samsung
Features: 3840x2160 (4K) native resolution OLED HDR 144Hz refresh rate w/ AMD FreeSync Tizen smart OS 4 HDMI inputs Model: QN42S90DAEXZA
URL: https://www.dealnews.com/products/Sams

In [10]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [11]:
result = get_recommendations()

In [12]:
len(result.deals)

5

In [13]:
result.deals[0]

Deal(product_description='The Samsung S90D Series QN42S90DAEXZA is a 42-inch OLED Smart TV that boasts a stunning 4K HDR resolution of 3840x2160, ensuring that every detail comes alive with vibrant colors and deep contrasts. It features a rapid 144Hz refresh rate, making it an ideal choice for fast-paced gaming and action-packed movies. This television runs on the Tizen smart OS, providing seamless access to streaming platforms and smart apps. With four HDMI inputs, connecting your devices is hassle-free.', price=1100.0, url='https://www.dealnews.com/products/Samsung/Samsung-S90-D-Series-QN42-S90-DAEXZA-42-4-K-HDR-144-Hz-OLED-Smart-TV/485596.html?iref=rss-c142')

In [14]:
from agents.scanner_agent_az import ScannerAgent

In [15]:
agent = ScannerAgent()
result = agent.scan()

In [16]:
result

DealSelection(deals=[Deal(product_description='The Samsung S90D Series QN42S90DAEXZA is a state-of-the-art 42" 4K HDR OLED Smart TV that offers breathtaking picture quality with its high dynamic range capabilities and an ultra-smooth 144Hz refresh rate. It features 3840x2160 native resolution, making it perfect for enjoying your favorite movies and games in stunning detail. Equipped with Tizen smart OS, users can easily access a multitude of streaming services and apps. The TV has four HDMI inputs, allowing for easy connectivity with various devices, ensuring an immersive viewing experience.', price=1100.0, url='https://www.dealnews.com/products/Samsung/Samsung-S90-D-Series-QN42-S90-DAEXZA-42-4-K-HDR-144-Hz-OLED-Smart-TV/485596.html?iref=rss-c142'), Deal(product_description='The Lensbaby Velvet 85 is a remarkable 85mm f/1.8 lens designed for Micro Four Thirds cameras, ideal for photographers looking to expand their creative toolkit. Its unique design allows for deliciously soft focus e